In [83]:
#main data file and import script for class group 6  project 1
# Author AO ottoaara@gmail.com. 952.693.1378 
# 5.5.23
# data source mpls open data https://opendata.minneapolismn.gov/datasets/cityoflakes::crime-data/explore?location=19.216823%2C-46.664577%2C4.00
# CSV option under Crime Data Header
import pandas as pd
import csv
import json
import matplotlib as pyplot
import geopandas as gpd
from shapely.geometry import Point, Polygon
#crime_data_main  = full data set (75 Megs so seperated out into groups for ease of dev)
#ran_crime_data_df is a light 1000 rows of data randomly choosen from crime_data_man to work with
#crime_data_2019 = data for 2019 
#crime_data_2020 ...
#crime_data_2021 ...
#crime_data_2022...
##crime_data_2023 = data for 2023 
filename = "Crime_Data.csv"
csvdatapath = "Data/" + filename

In [84]:
# download local from repo to desktop (clone to your desktop)
#this reads the csv file 
crime_data_main = pd.read_csv(csvdatapath)

In [85]:
#columns listed for ease of co-development use. 
crime_data_main.columns

Index(['X', 'Y', 'Type', 'Case_Number', 'Case_NumberAlt', 'Reported_Date',
       'Occurred_Date', 'NIBRS_Crime_Against', 'NIBRS_Group', 'NIBRS_Code',
       'Offense_Category', 'Offense', 'Problem_Initial', 'Problem_Final',
       'Address', 'Precinct', 'Neighborhood', 'Ward', 'Latitude', 'Longitude',
       'wgsXAnon', 'wgsYAnon', 'Crime_Count', 'OBJECTID'],
      dtype='object')

In [86]:
#checkign data types 
crime_data_main.dtypes

X                      float64
Y                      float64
Type                    object
Case_Number             object
Case_NumberAlt          object
Reported_Date           object
Occurred_Date           object
NIBRS_Crime_Against     object
NIBRS_Group             object
NIBRS_Code              object
Offense_Category        object
Offense                 object
Problem_Initial         object
Problem_Final           object
Address                 object
Precinct               float64
Neighborhood            object
Ward                   float64
Latitude               float64
Longitude              float64
wgsXAnon               float64
wgsYAnon               float64
Crime_Count              int64
OBJECTID                 int64
dtype: object

In [87]:
#get count of one column to prep for null value check 
crime_data_main["X"].count()

226120

In [88]:
# Remove rows containing all null values- cleaning data set
crime_data_clean = crime_data_main.dropna(how='any')

In [89]:
# removing rows with null values.  
crime_data_clean["X"].count()

151701

In [90]:
# checking values of crime category to reduce our data set down further to violent crimes 
crime_data_clean["Offense_Category"].unique()

array(['Subset of NIBRS Assault Offenses', 'Assault Offenses ',
       'Larceny/Theft Offenses ', 'Sex Offenses ',
       'Drug/Narcotic Offenses ', 'Robbery ', 'Fraud Offenses ',
       'Burglary/Breaking & Entering ', 'Motor Vehicle Theft ',
       'Stolen Property Offenses ', 'Weapon Law Violations ',
       'Counterfeiting/Forgery ', 'Extortion/Blackmail ',
       'Kidnapping/Abduction ', 'Embezzlement ',
       'Subset of NIBRS Robbery', 'Pornography/Obscene Material ',
       'Prostitution Offenses ', 'Arson ', 'Homicide Offenses ',
       'Destruction/Damage/Vandalism of Property ', 'Animal Cruelty ',
       'Human Trafficking ', 'Bribery ', 'Gambling Offenses ',
       'Gunshot Wound Victims', 'Shots Fired Calls'], dtype=object)

In [91]:
# reducing the data sets to only have violent crimes in them.  Note Included motor vehicle theft due to the
# high volumme of car-jackings occuring in mpls. 

crime_categories = [
    'Subset of NIBRS Assault Offenses',
    'Assault Offenses',
    'Larceny/Theft Offenses',
    'Sex Offenses',
    'Robbery',
    'Burglary/Breaking & Entering',
    'Motor Vehicle Theft',
    'Weapon Law Violations',
    'Kidnapping/Abduction',
    'Homicide Offenses',
    'Human Trafficking',
    'Gunshot Wound Victims',
    'Shots Fired Calls'
]

crime_data_clean = crime_data_clean[crime_data_clean['Offense_Category'].isin(crime_categories)]


In [92]:
#changing Occured_Date filed to data/time from object
crime_data_clean['Occurred_Date'] = pd.to_datetime(crime_data_clean['Occurred_Date'])


In [93]:
#checking to make sure date/time conv worked 
crime_data_clean.dtypes

X                                  float64
Y                                  float64
Type                                object
Case_Number                         object
Case_NumberAlt                      object
Reported_Date                       object
Occurred_Date          datetime64[ns, UTC]
NIBRS_Crime_Against                 object
NIBRS_Group                         object
NIBRS_Code                          object
Offense_Category                    object
Offense                             object
Problem_Initial                     object
Problem_Final                       object
Address                             object
Precinct                           float64
Neighborhood                        object
Ward                               float64
Latitude                           float64
Longitude                          float64
wgsXAnon                           float64
wgsYAnon                           float64
Crime_Count                          int64
OBJECTID   

In [94]:
#building random data set 
n_samples = min(1000, len(crime_data_clean))
ran_crime_data_df = crime_data_clean.sample(n=n_samples, random_state=42)

In [95]:
#building year data sets
crime_data_2019 = crime_data_clean[crime_data_clean['Occurred_Date'].dt.year == 2019]
crime_data_2020 = crime_data_clean[crime_data_clean['Occurred_Date'].dt.year == 2020]
crime_data_2021 = crime_data_clean[crime_data_clean['Occurred_Date'].dt.year == 2021]
crime_data_2022 = crime_data_clean[crime_data_clean['Occurred_Date'].dt.year == 2022]
crime_data_2023 = crime_data_clean[crime_data_clean['Occurred_Date'].dt.year == 2023]

In [96]:
# grouping with smaller data set to look at where the crimes are happening - counting number of offenses 
#grouped_data = crime_data_clearn.groupby(['Neighborhood', 'Address']).size().reset_index(name='Offense_Count')
grouped_data = crime_data_clean.groupby(['Neighborhood', 'Address', 'Offense_Category']).size().reset_index(name='Offense_Count')


In [97]:
#sorting grouped data to show the worst parts of town ie sorting by places more than one offense occurred
sorted_grouped_data = grouped_data.sort_values(by='Offense_Count', ascending=False)


In [98]:
# viewing sorted data frame. 
sorted_grouped_data

,Neighborhood,Address,Offense_Category,Offense_Count
953,Hawthorne,0021XX ALDRICH AVE N,Shots Fired Calls,112
952,Hawthorne,0021XX ALDRICH AVE N,Gunshot Wound Victims,66
563,East Phillips,0024XX 18TH AVE S,Subset of NIBRS Assault Offenses,60
886,Hawthorne,0004XX 29TH AVE N,Subset of NIBRS Assault Offenses,50
1793,McKinley,0035XX BRYANT AVE N,Subset of NIBRS Assault Offenses,47
...,...,...,...,...
1383,Lind - Bohanon,0047XX 6TH ST N,Gunshot Wound Victims,1
1379,Lind - Bohanon,0046XX FREMONT AVE N,Shots Fired Calls,1
1372,Lind - Bohanon,0046XX BRYANT AVE N,Gunshot Wound Victims,1
1371,Lind - Bohanon,0045XX LYNDALE AVE N,Shots Fired Calls,1


In [99]:
# adding in date time to support further analysis to check correlation between time and offense
grouped_data_bydate = crime_data_clean.groupby(['Neighborhood', 'Address', 'Offense_Category', 'Occurred_Date']).size().reset_index(name='Offense_Count')


In [100]:
grouped_data_bydate

,Neighborhood,Address,Offense_Category,Occurred_Date,Offense_Count
0,Armatage,0021XX 53RD ST W,Subset of NIBRS Assault Offenses,2019-07-08 15:29:00+00:00,1
1,Armatage,0027XX 55TH ST W,Subset of NIBRS Assault Offenses,2019-03-23 23:37:00+00:00,1
2,Armatage,0027XX 57TH ST W,Subset of NIBRS Assault Offenses,2019-02-27 19:00:00+00:00,1
3,Armatage,0027XX 57TH ST W,Subset of NIBRS Assault Offenses,2023-03-14 14:53:00+00:00,1
4,Armatage,0027XX 59TH ST W,Subset of NIBRS Assault Offenses,2020-05-19 18:20:00+00:00,1
...,...,...,...,...,...
10023,Windom Park,0020XX 19TH AVE NE,Subset of NIBRS Assault Offenses,2022-02-09 14:28:00+00:00,1
10024,Windom Park,0020XX 19TH AVE NE,Subset of NIBRS Assault Offenses,2023-03-02 00:00:00+00:00,1
10025,Windom Park,0022XX CENTRAL AVE NE,Gunshot Wound Victims,2021-07-31 05:34:00+00:00,1
10026,Windom Park,0022XX CENTRAL AVE NE,Shots Fired Calls,2021-07-31 05:34:55+00:00,1


In [101]:
# viewing sorted data frame. 
sorted_grouped_data = grouped_data_bydate.sort_values(by='Offense_Count', ascending=False)
sorted_grouped_data

,Neighborhood,Address,Offense_Category,Occurred_Date,Offense_Count
6475,Near - North,0011XX NEWTON AVE N,Subset of NIBRS Assault Offenses,2023-02-02 17:11:00+00:00,2
2922,Hawthorne,0008XX 21ST AVE N,Shots Fired Calls,2022-11-09 20:16:35+00:00,2
0,Armatage,0021XX 53RD ST W,Subset of NIBRS Assault Offenses,2019-07-08 15:29:00+00:00,1
6684,Near - North,0016XX DUPONT AVE N,Subset of NIBRS Assault Offenses,2019-09-16 14:47:00+00:00,1
6685,Near - North,0016XX DUPONT AVE N,Subset of NIBRS Assault Offenses,2019-09-18 20:38:00+00:00,1
...,...,...,...,...,...
3344,Hawthorne,0022XX DUPONT AVE N,Shots Fired Calls,2021-05-27 00:41:34+00:00,1
3345,Hawthorne,0022XX EMERSON AVE N,Gunshot Wound Victims,2022-09-04 01:14:00+00:00,1
3346,Hawthorne,0022XX EMERSON AVE N,Shots Fired Calls,2022-03-20 04:28:50+00:00,1
3347,Hawthorne,0022XX EMERSON AVE N,Shots Fired Calls,2022-09-04 01:14:58+00:00,1


In [102]:
# adding long and lat. for analysis by location 
crime_data_byLongLat = grouped_data_bydate.merge(
    crime_data_clean[['Neighborhood', 'Address', 'Latitude', 'Longitude']],
    on=['Neighborhood', 'Address'],
    how='left')

In [103]:
crime_data_bylonglat

,X,Y,Type,Case_Number,Case_NumberAlt,Reported_Date,Occurred_Date,NIBRS_Crime_Against,NIBRS_Group,NIBRS_Code,...,Neighborhood,Ward,Latitude_x,Longitude_x,wgsXAnon,wgsYAnon,Crime_Count,OBJECTID,Latitude_y,Longitude_y
0,-93.249966,44.970026,Additional Crime Metrics,23-086158,MP202386158,2023/04/09 15:46:00+00,2023-04-09 14:50:00+00:00,Non NIBRS Data,Non NIBRS Data,Non NIBRS Data,...,Cedar Riverside,6.0,44.97003,-93.24997,-1.038054e+07,5.616804e+06,3,6093,44.97003,-93.24997
1,-93.249966,44.970026,Additional Crime Metrics,23-086158,MP202386158,2023/04/09 15:46:00+00,2023-04-09 14:50:00+00:00,Non NIBRS Data,Non NIBRS Data,Non NIBRS Data,...,Cedar Riverside,6.0,44.97003,-93.24997,-1.038054e+07,5.616804e+06,3,6093,44.97003,-93.24997
2,-93.249966,44.970026,Additional Crime Metrics,23-086158,MP202386158,2023/04/09 15:46:00+00,2023-04-09 14:50:00+00:00,Non NIBRS Data,Non NIBRS Data,Non NIBRS Data,...,Cedar Riverside,6.0,44.97003,-93.24997,-1.038054e+07,5.616804e+06,3,6093,44.97003,-93.24997
3,-93.249966,44.970026,Additional Crime Metrics,23-086158,MP202386158,2023/04/09 15:46:00+00,2023-04-09 14:50:00+00:00,Non NIBRS Data,Non NIBRS Data,Non NIBRS Data,...,Cedar Riverside,6.0,44.97003,-93.24997,-1.038054e+07,5.616804e+06,3,6093,44.97003,-93.24997
4,-93.249966,44.970026,Additional Crime Metrics,23-086158,MP202386158,2023/04/09 15:46:00+00,2023-04-09 14:50:00+00:00,Non NIBRS Data,Non NIBRS Data,Non NIBRS Data,...,Cedar Riverside,6.0,44.97003,-93.24997,-1.038054e+07,5.616804e+06,3,6093,44.97003,-93.24997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26322,-93.315244,45.007836,Gunshot Wound Victims,19-262832,MP2019262832,2019/08/31 19:47:00+00,2019-08-31 18:28:00+00:00,Non NIBRS Data,Non NIBRS Data,Non NIBRS Data,...,Jordan,4.0,45.00784,-93.31524,-1.038781e+07,5.622755e+06,1,215643,45.00784,-93.31524
26323,-93.315244,45.007836,Gunshot Wound Victims,19-262832,MP2019262832,2019/08/31 19:47:00+00,2019-08-31 18:28:00+00:00,Non NIBRS Data,Non NIBRS Data,Non NIBRS Data,...,Jordan,4.0,45.00784,-93.31524,-1.038781e+07,5.622755e+06,1,215643,45.00784,-93.31524
26324,-93.315244,45.007836,Gunshot Wound Victims,19-262832,MP2019262832,2019/08/31 19:47:00+00,2019-08-31 18:28:00+00:00,Non NIBRS Data,Non NIBRS Data,Non NIBRS Data,...,Jordan,4.0,45.00784,-93.31524,-1.038781e+07,5.622755e+06,1,215643,45.00784,-93.31524
26325,-93.315244,45.007836,Gunshot Wound Victims,19-262832,MP2019262832,2019/08/31 19:47:00+00,2019-08-31 18:28:00+00:00,Non NIBRS Data,Non NIBRS Data,Non NIBRS Data,...,Jordan,4.0,45.00784,-93.31524,-1.038781e+07,5.622755e+06,1,215643,45.00784,-93.31524
